# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [2]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()

API key looks good so far


In [3]:
system_prompt = "You are an expert that analyzes the python code. \
Answer the questions in a way that is to the point and not too long."

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why in Korean:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Get gpt-4o-mini to answer, with streaming
def stream_explain():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ],
        stream=True
    )

    response = ""
    # Markdown(""): 빈 마크다운 객체를 생성 (초기에는 빈 내용)
    # display_id=True: 이 출력에 고유한 ID를 부여해서 나중에 업데이트할 수 있게 함
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''  # 새로운 텍스트 조각 누적
        response = response.replace("```","").replace("markdown", "")  # 마크다운 코드 블록 제거
        update_display(Markdown(response), display_id=display_handle.display_id)

In [6]:
stream_explain()

이 코드는 `books`라는 목록에서 각 책 객체의 저자 정보를 추출하여 생성하는 제너레이터 표현식입니다. 

1. `{book.get("author") for book in books if book.get("author")}`: 이 부분은 저자가 있는 책의 저자 이름을 집합(set)으로 만듭니다. 집합을 사용함으로써 중복된 저자 이름은 제거됩니다.
2. `yield from`: 이 구문은 생성자 함수에서 다른 생성자나 이터레이터의 값을 순차적으로 반환합니다. 

결과적으로 이 코드는 `books` 목록의 모든 책에서 저자를 추출하여 중복 없이 저자 목록을 생성하는 역할을 합니다.

In [22]:
# Get Llama 3.2 to answer
def normal_explain():
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ],
    )
    result = response['message']['content']
    display(Markdown(result))

In [23]:
normal_explain()

**해석:**

이 코드는 livro서의 저자 정보를 가져오고, livro서의 정보를 순환합니다.

* `yield from`는 서브루틴 안에서 다른 루outine의 결과물을 순환하도록 하는 명령입니다.
* `{book.get("author") for book in books if book.get("author")}`는 list comprehension으로 책의 author 정보를 가져오고, 만약 author가 없는 경우이든 or author 정보를 가져올 수 없는 경우이든 Then 그 Book을 무시합니다. 

예를 들어:

* `books`에는 [{"title": "책1", "author": "저자1"}, {"title": "책2", "author": ""}, {"title": "책3", "author": "저자2"}]这样的 book 정보가 있습니다.
* 이 코드는 author 정보가 있는 book을 순환합니다. 

**사용 예:**

```
def fetch_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}
    
books = [{"title": "책1", "author": "저자1"}, {"title": "책2", "author": ""}, {"title": "책3", "author": "저자2"}]
for author in fetch_authors(books):
    print(author)
```